# 유튜브 스크립트 불러오기

pip install langchain

In [1]:
from langchain.document_loaders import YoutubeLoader

pip install youtube_transcript_api

In [13]:
loader = YoutubeLoader.from_youtube_url("https://www.youtube.com/watch?v=vQT9gL0Fack")
transcript = loader.load()
transcript

[Document(page_content="today we have Elon Musk Elon thank you for joining us yeah thanks for having me so we want to spend the time today talking about uh your view of the future and what people should work on so to start off could you tell us you famously said when you were younger there were five problems that you thought were most important for you to work on um if you were 22 today what would the five problems that you would think about working on B uh well first of all I think um if somebody is doing something that is useful to the rest of society I think that's a good thing like it doesn't have to change the world like you know um if you're doing something that has high value to people um and frankly even if it's something if it's like um just a little game or you know the some improvement in photo sharing or something if it if it has a small amount of good uh for a large number of people um that's I mean I think that's that's fine like stuff doesn't need to be changed the world

# 긴 내용 요약하기

In [14]:
from langchain.prompts import PromptTemplate
from langchain.chains.summarize import load_summarize_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chat_models import ChatOpenAI

## 텍스트 쪼개기

In [15]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=4000, chunk_overlap=0)
text = text_splitter.split_documents(transcript)

In [16]:
len(text)

5

In [17]:
text[0]

Document(page_content="today we have Elon Musk Elon thank you for joining us yeah thanks for having me so we want to spend the time today talking about uh your view of the future and what people should work on so to start off could you tell us you famously said when you were younger there were five problems that you thought were most important for you to work on um if you were 22 today what would the five problems that you would think about working on B uh well first of all I think um if somebody is doing something that is useful to the rest of society I think that's a good thing like it doesn't have to change the world like you know um if you're doing something that has high value to people um and frankly even if it's something if it's like um just a little game or you know the some improvement in photo sharing or something if it if it has a small amount of good uh for a large number of people um that's I mean I think that's that's fine like stuff doesn't need to be changed the world 

In [18]:
text[1]

Document(page_content="they're at a steep point in the inflection code and um and I didn't want to you know to appears to Stanford and then and what should all happen um and then and I wasn't entirely certain that the technology I'd be working on would actually succeed um like you can get you can get a you know Doctrine on many things that ultimately are not do not have a practical bearing on the world um and I wanted to you know just I really was just trying to be useful that's the optimization it's like what what are the what can I do that would actually be useful do you think people that want to be useful today should get phds um mostly not someone what is the best way some yes but mostly not um how should someone figure out how they can be most useful whatever this thing is that you're trying to create what would what would be the utility Delta compared to the current state of the art times how many people it would affect so that's why I think um having something that has a that's 

## 사용할 LLM 모델 설정

pip install openai
pip install tiktoken

In [21]:
llm = ChatOpenAI(temperature=0,
                    openai_api_key="api key",
                    max_tokens=3000,
                    model_name="gpt-3.5-turbo",
                    request_timeout=120
                )

## 요약에 사용할 프롬프트

In [22]:
# 각각의 chunck 를 요약하기
prompt = PromptTemplate(
    template="""Summarize the youtube video whose transcript is provided within backticks \
    ```{text}```
    """, input_variables=["text"]
)

# 요약된 내용들을 취합하여 다시한번 요약하기
combine_prompt = PromptTemplate(
    template="""Combine all the youtube video transcripts  provided within backticks \
    ```{text}```
    Provide a concise summary between 8 to 10 sentences.
    """, input_variables=["text"]
)

## 요약 시작하기

In [24]:
chain = load_summarize_chain(llm, chain_type="map_reduce", verbose=False, map_prompt=prompt, combine_prompt=combine_prompt)

In [26]:
output = chain.run(text)

In [27]:
output

"Elon Musk discusses various important issues in the provided YouTube video transcripts. He emphasizes the value of working on something useful to society, even if it doesn't change the world. Musk identifies AI as the most significant issue that will impact humanity, stressing the need to ensure its development is beneficial. He also highlights genetics as a crucial problem to solve, particularly in preventing diseases like Dementia and Alzheimer's. Additionally, Musk emphasizes the need for a high bandwidth interface between the brain and digital technology. He reflects on his own journey, mentioning his initial focus on electrification of cars and energy storage technologies. \n\nThe videos also discuss the importance of making a big impact and creating something that makes a difference. The speaker talks about their decision to start SpaceX, despite it being considered crazy at the time, and criticizes big aerospace companies for not focusing on radical innovation. They highlight t

## 번역하기

pip install googletrans==3.1.0a0

In [29]:
from googletrans import Translator

def google_trans(messages):
    google = Translator()
    result = google.translate(messages, dest="ko")

    return result.text

result = google_trans(output)
print(result)

Elon Musk는 제공된 YouTube 비디오 대본에서 다양한 중요한 문제에 대해 논의합니다. 세상을 바꾸지는 못하더라도 사회에 도움이 되는 일을 하는 것의 가치를 강조한다. 머스크는 AI를 인류에게 영향을 미칠 가장 중요한 문제로 꼽으며 AI의 발전이 유익하도록 보장해야 할 필요성을 강조했습니다. 그는 또한 특히 치매 및 알츠하이머병과 같은 질병을 예방하는 데 있어 해결해야 할 중요한 문제로 유전학을 강조합니다. 또한 Musk는 두뇌와 디지털 기술 간의 고대역폭 인터페이스의 필요성을 강조합니다. 그는 자동차의 전기화와 에너지 저장 기술에 대한 초기 초점을 언급하면서 자신의 여정을 되돌아봅니다.

비디오는 또한 큰 영향을 미치고 차이를 만드는 무언가를 만드는 것의 중요성에 대해 논의합니다. 연사는 당시 미친 짓으로 여겨졌음에도 불구하고 SpaceX를 시작하기로 한 결정에 대해 이야기하고 급진적 혁신에 집중하지 않는 대형 항공 우주 회사를 비판합니다. 그들은 기술 개선 노력 부족으로 인한 문명의 쇠퇴를 강조하고 똑똑한 사람들이 기술을 개선하기 위해 지속적으로 노력해야 할 필요성을 강조합니다. 비디오는 또한 다른 사람들이 아이디어를 미쳤다고 생각할 때에도 아이디어를 추구하는 머스크의 대담성을 다룹니다.

머스크는 또한 AI 기술의 민주화 필요성과 두려움에도 불구하고 조치를 취하는 것의 중요성에 대해 논의합니다. 그는 SpaceX와 Tesla를 시작할 때 초기 의심과 두려움을 공유하지만 가능성을 받아들이고 앞으로 나아가는 것의 중요성을 강조합니다. 머스크는 자급자족할 수 있는 화성 식민지를 건설할 가능성과 SpaceX의 생존을 보장해야 할 필요성에 대한 자신의 믿음을 표현합니다. 그는 또한 AI 기술의 잠재적 위험에 대해 논의하고 단일 주체의 통제를 방지하기 위해 AI 기술의 민주화를 옹호합니다.

또한 비디오는 OpenAI에 대한 Musk의 참여와 AI 기술을 확산하려는 임무에 대해 설명합니다. 그는 AI와 효과적으로 병합하고 AI-인간 공생체를 만들기 위해 고